In [3]:
%pip install duckdb


Note: you may need to restart the kernel to use updated packages.


In [4]:
import duckdb

1. Create a connection and a cursor

In [5]:
conn = duckdb.connect("database.db")
conn

In [6]:
cur = conn.cursor()

Now let's do something with our curson

In [7]:
cur.execute("SELECT * FROM Site LIMIT 5")

Now we want to see our results...there are 3 ways of getting them
1. All results at once 

In [8]:
# this prints out a list with each set of values as a tuple
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375),
 ('cakr',
  'Cape Krusenstern',
  'Alaska, USA',
  67.0999984741211,
  -163.5,
  54.099998474121094),
 ('cari',
  'Canning River Delta',
  'Alaska, USA',
  70.0999984741211,
  -145.8000030517578,
  722.0)]

Cursors do not store anything, they just transfer queries to the database and get results back 
Therefore if we were to re-run just `cur.fetchall()` we would return an empty list 
You will always get back tuples even if you only request 1 columns

In [10]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")

In [11]:
# returns a tuple for each query
cur.fetchall()

[('14HPE1',),
 ('11eaba',),
 ('11eabaagc01',),
 ('11eabaagv01',),
 ('11eababbc02',),
 ('11eababsv01',),
 ('11eabaduh01',),
 ('11eabaduv01',),
 ('11eabarpc01',),
 ('11eabarpc02',)]

In [13]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
# list comprehension; for each tuple t give me the 1st element in cur.fetchall
# returns a string for each query
[t[0] for t in cur.fetchall()]

['14HPE1',
 '11eaba',
 '11eabaagc01',
 '11eabaagv01',
 '11eababbc02',
 '11eababsv01',
 '11eabaduh01',
 '11eabaduv01',
 '11eabarpc01',
 '11eabarpc02']

2. Get the one result, or the next result

In [15]:
# returns a tuple
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchall()

[(1547,)]

In [14]:
# returns a list
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()

(1547,)

In [16]:
# returns just the number
cur.execute("SELECT COUNT(*) FROM Bird_nests")
cur.fetchone()[0]

1547

3. Using an iterator - but DuckDB doesn't support iterators

In [17]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
for row in cur:
    print(f"got {row[0]}")
            

TypeError: 'duckdb.duckdb.DuckDBPyConnection' object is not iterable

In [20]:
cur.execute("SELECT Nest_ID FROM Bird_nests LIMIT 10")
while True:
    row = cur.fetchone()
    if row == None:
        break
    # do something with row
    print(f"got nest ID {row[0]}")

got nest ID 14HPE1
got nest ID 11eaba
got nest ID 11eabaagc01
got nest ID 11eabaagv01
got nest ID 11eababbc02
got nest ID 11eababsv01
got nest ID 11eabaduh01
got nest ID 11eabaduv01
got nest ID 11eabarpc01
got nest ID 11eabarpc02


Can use statements other than SELECT as well

In [21]:
# """ a way of quoting a multi-line string
cur.execute(""" 
            CREATE TEMP TABLE temp_table AS
            SELECT * FROM Bird_nests LIMIT 10
            """)

In [22]:
cur.execute("SELECT * FROM temp_table")

In [23]:
cur.fetchall()

[('b14.6',
  2014,
  'chur',
  '14HPE1',
  'sepl',
  'vloverti',
  datetime.date(2014, 6, 14),
  None,
  3,
  None,
  None),
 ('b11.7',
  2011,
  'eaba',
  '11eaba',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 10),
  'searcher',
  4,
  None,
  None),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagc01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  6.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaagv01',
  'amgp',
  'dkessler',
  datetime.date(2011, 6, 25),
  'searcher',
  3,
  3.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eababbc02',
  'bbpl',
  'dkessler',
  datetime.date(2011, 6, 24),
  'searcher',
  4,
  4.0,
  'float'),
 ('b11.7',
  2011,
  'eaba',
  '11eababsv01',
  'wrsa',
  'bhill',
  datetime.date(2011, 7, 7),
  'searcher',
  4,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduh01',
  'dunl',
  'dkessler',
  datetime.date(2011, 6, 28),
  'searcher',
  3,
  2.0,
  'float'),
 ('b11.6',
  2011,
  'eaba',
  '11eabaduv01',
  'dunl',
  'dk

A note on fragility

For example:
INSERT INTO Site VALUES ("abcd", "Foo", 35.7, -119.5, "?")

A less fragile way of expressing the same thing:
INSERT INTO Site (Code, Site_name, Latitude, Longitude, Something_else)
    VALUES ("abcd", "Foo", 35.7, -119.5, "?")

In the same vein: SELECT * is fragile 

In [24]:
cur.execute("SELECT * FROM Site LIMIT 3")
cur.fetchall()

[('barr',
  'Barrow',
  'Alaska, USA',
  71.30000305175781,
  -156.60000610351562,
  220.39999389648438),
 ('burn',
  'Burntpoint Creek',
  'Ontario, Canada',
  55.20000076293945,
  -84.30000305175781,
  63.0),
 ('bylo',
  'Bylot Island',
  'Nunavut, Canada',
  73.19999694824219,
  -80.0,
  723.5999755859375)]

A better more robust way to code something

In [25]:
# a better way
cur.execute("SELECT Site_name, Code, Latitude, Longitude FROM Site LIMIT 3")
cur.fetchall()

[('Barrow', 'barr', 71.30000305175781, -156.60000610351562),
 ('Burntpoint Creek', 'burn', 55.20000076293945, -84.30000305175781),
 ('Bylot Island', 'bylo', 73.19999694824219, -80.0)]

Question we are trying to answer: How many nests do we have for each species?

Approach: first get all species. Then execute a count query for each species

This approach is rather dangerous actually

In [35]:
query = """
SELECT COUNT(*) FROM Bird_nests
WHERE Species = '%s'
"""

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall(): #DuckDB work around
    code = row[0]
    prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(prepared_query)
    print(f"Species {code} has {cur2.fetchone()[0]} nests")

Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Side note 

In [30]:
s = "My name is %s"
print(s % "greg")

# and 
s = "My name is %s and the other teachers name is %s"
print(s % ("greg", "julian"))

# the new f-string method
name = "greg"
print(f"My name is {name}")

#third way
print("My name is {}".format("Greg"))


My name is greg
My name is greg and the other teachers name is julian
My name is greg
My name is Greg


The above Python interpolation is dangerous and has caused many database hacks! There is a better way

In [39]:
query = """
SELECT COUNT(*) FROM Bird_nests
WHERE Species = ? 
"""

cur.execute("SELECT Code FROM Species LIMIT 3")
for row in cur.fetchall(): #DuckDB work around
    code = row[0]
    #Not needed: prepared_query = query % code
    #print(prepared_query)
    cur2 = conn.cursor()
    cur2.execute(query, [code]) # add argument here
    print(f"Species {code} has {cur2.fetchone()[0]} nests")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Species agsq has 0 nests
Species amcr has 0 nests
Species amgp has 29 nests


Let's illustrate the danger with a different example

In [40]:
abbrev = "TS"
name = "Taylor Swift"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')
            """ % (abbrev, name))


In [41]:
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('emagnuson', 'Emily Magnuson'),
 ('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift')]

In [42]:
abbrev = "CO"
name = "Conan O'Brian"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')
            """ % (abbrev, name))

ParserException: Parser Error: syntax error at or near "Brian"

In [44]:
"""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES ('%s', '%s')
            """ % (abbrev, name)

"\n            INSERT INTO Personnel (Abbreviation, Name)\n            VALUES ('CO', 'Conan O'Brian')\n            "

errors due to the ' in the last name

In [45]:
abbrev = "CO"
name = "Conan O'Brian"
cur.execute("""
            INSERT INTO Personnel (Abbreviation, Name)
            VALUES (?, ?)
            """,
            [abbrev, name])

In [46]:
cur.execute("SELECT * FROM Personnel")
cur.fetchall()[-3:]

[('mcorrell', 'Maureen Correll'),
 ('TS', 'Taylor Swift'),
 ('CO', "Conan O'Brian")]